# Averaged Density-Speed Analysis Across Simulations

This notebook aggregates `output_1` ... `output_10` (or any `output_<int>` folders),
aligns them to a common time horizon, bins by mean density, averages normalized street
speeds across all compatible realizations, saves mean/error matrices, and reruns the same
class analysis and plots from `minimal_density_speed.ipynb` on averaged data.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

ROOT = Path("..")
OUTPUT_DIR = ROOT / "output_avg"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_SEED = 42
TIME_COL = "time_step"
DENSITY_COL = "mean_density_vpk"


def discover_simulation_dirs(root: Path):
    dirs = []
    for p in root.iterdir():
        if not p.is_dir() or not p.name.startswith("output_"):
            continue
        suffix = p.name.split("_", 1)[1]
        if suffix.isdigit():
            dirs.append(p)
    return sorted(dirs, key=lambda x: int(x.name.split("_")[1]))


sim_dirs = discover_simulation_dirs(ROOT)
if not sim_dirs:
    raise RuntimeError("No output_<int> simulation folders found.")

print(f"Discovered simulations: {[p.name for p in sim_dirs]}")
print(f"Aggregation output folder: {OUTPUT_DIR}")


In [ ]:
# Validate required inputs
required_files = ["data.csv", "stability_timestep.csv", "normalized_street_speeds.csv"]
missing = []

for d in sim_dirs:
    for fn in required_files:
        if not (d / fn).exists():
            missing.append(str(d / fn))

if missing:
    raise FileNotFoundError(
        "Missing required files:\n" + "\n".join(missing)
    )

print("All required input files are present.")


In [ ]:
# Compute per-simulation last timestep from data.csv and common truncation limit
last_rows = []

for d in sim_dirs:
    data = pd.read_csv(d / "data.csv", sep=";").sort_values(TIME_COL)
    if data.empty:
        continue
    last_rows.append(
        {
            "simulation": d.name,
            "last_time_step": float(data[TIME_COL].iloc[-1]),
            "samples": int(len(data)),
        }
    )

if not last_rows:
    raise RuntimeError("No data rows found in simulation folders.")

last_timestep_df = pd.DataFrame(last_rows).sort_values("simulation")
common_time_limit = int(last_timestep_df["last_time_step"].min())

limit_path = OUTPUT_DIR / "simulation_last_timesteps.csv"
last_timestep_df.to_csv(limit_path, index=False, sep=";")

print(last_timestep_df)
print(f"Common truncation limit (min last timestep): {common_time_limit}")
print(f"Saved per-run timestep summary: {limit_path}")


In [ ]:
# Pick one random stability_timestep.csv and use its step count (<= common limit) as number of bins
rng = np.random.default_rng(RANDOM_SEED)
reference_dir = sim_dirs[int(rng.integers(0, len(sim_dirs)))]

reference_stability = (
    pd.read_csv(reference_dir / "stability_timestep.csv", sep=";")
    .sort_values(TIME_COL)
    .reset_index(drop=True)
)
reference_truncated = reference_stability[reference_stability[TIME_COL] <= common_time_limit]
n_bins = int(len(reference_truncated))

if n_bins <= 0:
    raise RuntimeError(
        f"Reference stability file {reference_dir/'stability_timestep.csv'} has no rows at or before time limit {common_time_limit}."
    )

print(f"Random reference simulation: {reference_dir.name}")
print(f"Reference stability rows up to limit: {n_bins}")


In [ ]:
# Load and truncate normalized speed matrices, then define constant-width mean-density bins
normalized_runs = []
pooled_density = []

for d in sim_dirs:
    df = (
        pd.read_csv(d / "normalized_street_speeds.csv", sep=";")
        .sort_values(TIME_COL)
        .reset_index(drop=True)
    )
    df = df[df[TIME_COL] <= common_time_limit].copy()
    df["simulation"] = d.name

    normalized_runs.append(df)
    pooled_density.append(df[DENSITY_COL].dropna())

if not pooled_density:
    raise RuntimeError("No density data available after truncation.")

all_density = pd.concat(pooled_density, ignore_index=True)
density_min = float(all_density.min())
density_max = float(all_density.max())

if density_min == density_max:
    bin_edges = np.array([density_min, density_max + 1e-9])
    n_bins = 1
else:
    bin_edges = np.linspace(density_min, density_max, n_bins + 1)

bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0

bins_df = pd.DataFrame(
    {
        "bin_id": np.arange(1, len(bin_centers) + 1),
        "density_low": bin_edges[:-1],
        "density_high": bin_edges[1:],
        "density_midpoint": bin_centers,
    }
)
bins_path = OUTPUT_DIR / "density_bins.csv"
bins_df.to_csv(bins_path, index=False, sep=";")

print(f"Density range: [{density_min:.4f}, {density_max:.4f}]")
print(f"Number of bins: {len(bin_centers)}")
print(f"Saved bins: {bins_path}")
bins_df.head()


In [ ]:
# Aggregate means/stds of normalized street speeds for each density bin
ignore_cols = {"datetime", TIME_COL, DENSITY_COL, "simulation"}
street_cols = sorted(
    set().union(*[{c for c in df.columns if c not in ignore_cols} for df in normalized_runs])
)

mean_rows = []
err_rows = []
bin_meta_rows = []

for i, (lo, hi, mid) in enumerate(zip(bin_edges[:-1], bin_edges[1:], bin_centers), start=1):
    per_run_matches = []
    for df in normalized_runs:
        if i < len(bin_centers):
            mask = (df[DENSITY_COL] >= lo) & (df[DENSITY_COL] < hi)
        else:
            mask = (df[DENSITY_COL] >= lo) & (df[DENSITY_COL] <= hi)
        matched = df.loc[mask]
        if not matched.empty:
            per_run_matches.append(matched)

    if per_run_matches:
        matched_rows = pd.concat(per_run_matches, ignore_index=True, sort=False)
    else:
        matched_rows = pd.DataFrame(columns=[DENSITY_COL, *street_cols])

    mean_row = {TIME_COL: i, DENSITY_COL: mid}
    err_row = {TIME_COL: i, DENSITY_COL: mid}

    density_vals = pd.to_numeric(matched_rows.get(DENSITY_COL), errors="coerce").dropna()
    if len(density_vals) > 0:
        # RMS spread around the bin midpoint (requested density error wrt midpoint)
        density_std_midpoint = float(np.sqrt(np.mean((density_vals - mid) ** 2)))
        density_std_sample = float(density_vals.std(ddof=1)) if len(density_vals) > 1 else np.nan
    else:
        density_std_midpoint = np.nan
        density_std_sample = np.nan

    err_row["density_std_midpoint"] = density_std_midpoint
    err_row["density_std_sample"] = density_std_sample

    for col in street_cols:
        if col in matched_rows.columns:
            vals = pd.to_numeric(matched_rows[col], errors="coerce").dropna()
        else:
            vals = pd.Series(dtype=float)

        mean_row[col] = float(vals.mean()) if len(vals) > 0 else np.nan
        err_row[col] = float(vals.std(ddof=1)) if len(vals) > 1 else np.nan

    mean_rows.append(mean_row)
    err_rows.append(err_row)
    bin_meta_rows.append(
        {
            "bin_id": i,
            "density_low": lo,
            "density_high": hi,
            "density_midpoint": mid,
            "matched_rows": int(len(matched_rows)),
            "matched_simulations": int(matched_rows["simulation"].nunique()) if "simulation" in matched_rows.columns and not matched_rows.empty else 0,
        }
    )

avg_normalized_df = pd.DataFrame(mean_rows)
avg_errors_df = pd.DataFrame(err_rows)
bin_meta_df = pd.DataFrame(bin_meta_rows)

avg_path = OUTPUT_DIR / "normalized_street_speeds.csv"
err_path = OUTPUT_DIR / "normalized_street_speeds_errors.csv"
meta_path = OUTPUT_DIR / "aggregation_bin_summary.csv"

avg_normalized_df.to_csv(avg_path, index=False, sep=";")
avg_errors_df.to_csv(err_path, index=False, sep=";")
bin_meta_df.to_csv(meta_path, index=False, sep=";")

print(f"Street columns aggregated: {len(street_cols)}")
print(f"Saved averaged matrix: {avg_path}")
print(f"Saved error matrix: {err_path}")
print(f"Saved bin aggregation summary: {meta_path}")
avg_normalized_df.head()


In [ ]:
# Plot averaged normalized speed curves vs binned mean density
keep_streets = [c for c in street_cols if c in avg_normalized_df.columns and avg_normalized_df[c].notna().any()]

fig, ax = plt.subplots(figsize=(20, 12))
for col in keep_streets:
    series = avg_normalized_df[col]
    mask = series.notna() & avg_normalized_df[DENSITY_COL].notna()
    if mask.sum() < 2:
        continue
    ax.plot(
        avg_normalized_df[DENSITY_COL][mask],
        series[mask],
        marker="o",
        markersize=2,
        linewidth=0.8,
        alpha=0.35,
    )

ax.set_xlabel("Mean density (veh/km)")
ax.set_ylabel("Speed / maxspeed (ratio)")
ax.set_xlim(0, 20)
ax.set_title("Averaged normalized street speeds vs mean density")
ax.grid(True, which="both", linestyle="--", alpha=0.4)
plt.tight_layout()

plot_path = OUTPUT_DIR / "norm_speeds_vs_density.png"
plt.savefig(plot_path, dpi=500)
print(f"Saved plot: {plot_path}")
plt.show()


In [ ]:
# Same class analysis as minimal_density_speed, now on averaged data
WINDOW_BAND = 2.5
DENSITY_START = 2
DENSITY_END = 20

windows = []
low = DENSITY_START
high = low + WINDOW_BAND
while high <= DENSITY_END + 1e-9:
    windows.append((low, min(high, DENSITY_END + 1e-9)))
    high += WINDOW_BAND

classes = {f"{lo:.2f}-{hi:.2f}": [] for lo, hi in windows}

for col in keep_streets:
    ratio = avg_normalized_df[col]
    for lo, hi in windows:
        label = f"{lo:.2f}-{hi:.2f}"
        mask = (avg_normalized_df[DENSITY_COL] >= lo) & (avg_normalized_df[DENSITY_COL] < hi)
        vals = ratio[mask].dropna()
        if vals.empty:
            continue
        hi_idx = vals[vals > 0.55].index.min()
        if pd.isna(hi_idx):
            continue
        later = vals.loc[vals.index > hi_idx]
        if (later < 0.35).any():
            classes[label].append(col)
            break

classes = {k: v for k, v in classes.items() if v}
print(f"Classes found: {len(classes)}")
for lbl, streets in classes.items():
    print(f"  {lbl}: {len(streets)} streets")


In [ ]:
# Plot classes on the map with different colors
edges = pd.read_csv(ROOT / "input" / "edges_tl.csv", sep=";")

if not classes:
    print("No classes found; adjust WINDOW_BAND / thresholds.")
else:
    geom_map = edges.set_index(edges["id"].astype(str))["geometry"].to_dict()

    def parse_linestring(wkt):
        if not isinstance(wkt, str) or "LINESTRING" not in wkt:
            return None
        inner = wkt.split("(", 1)[1].rsplit(")", 1)[0]
        coords = []
        for pair in inner.split(','):
            parts = pair.strip().split()
            if len(parts) != 2:
                continue
            x, y = map(float, parts)
            coords.append((x, y))
        return coords if coords else None

    colors = plt.cm.tab20.colors
    fig, ax = plt.subplots(figsize=(10, 10))

    for sid, geom in geom_map.items():
        coords = parse_linestring(geom)
        if not coords:
            continue
        xs, ys = zip(*coords)
        ax.plot(xs, ys, color="lightgray", linewidth=0.6, alpha=0.3)

    for idx, (label, streets) in enumerate(classes.items()):
        color = colors[idx % len(colors)]
        for sid in streets:
            geom = geom_map.get(str(sid))
            coords = parse_linestring(geom)
            if not coords:
                continue
            xs, ys = zip(*coords)
            ax.plot(
                xs,
                ys,
                color=color,
                linewidth=2.0,
                alpha=0.9,
                label=label if idx == list(classes.keys()).index(label) else None,
            )

    handles, labels = ax.get_legend_handles_labels()
    unique = dict(zip(labels, handles))
    if unique:
        ax.legend(unique.values(), unique.keys(), title="Density window")
    ax.set_title("Streets with ratio drop (>0.55 then <0.35) within density windows")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_aspect("equal")

    map_path = OUTPUT_DIR / "drop_speeds_map.png"
    plt.savefig(map_path, dpi=500)
    plt.tight_layout()
    print(f"Saved plot: {map_path}")
    plt.show()


In [ ]:
# Plot averaged normalized speed ratios with class highlights
if not classes:
    print("No classes available; run the classification cell or adjust thresholds.")
else:
    class_streets = {s for streets in classes.values() for s in streets}
    colors = plt.cm.tab20.colors

    fig, ax = plt.subplots(figsize=(12, 8))

    for col in keep_streets:
        if col in class_streets:
            continue
        series = avg_normalized_df[col]
        mask = series.notna() & avg_normalized_df[DENSITY_COL].notna()
        if mask.sum() < 2:
            continue
        ax.plot(
            avg_normalized_df[DENSITY_COL][mask],
            series[mask],
            color="lightgray",
            marker="o",
            markersize=2,
            linewidth=0.8,
            alpha=0.4,
        )

    for idx, (label, streets) in enumerate(classes.items()):
        color = colors[idx % len(colors)]
        for col in streets:
            if col not in avg_normalized_df.columns:
                continue
            series = avg_normalized_df[col]
            mask = series.notna() & avg_normalized_df[DENSITY_COL].notna()
            if mask.sum() < 2:
                continue
            ax.plot(
                avg_normalized_df[DENSITY_COL][mask],
                series[mask],
                color=color,
                marker="o",
                markersize=2.5,
                linewidth=1.0,
                alpha=0.9,
                label=label,
            )

    handles, labels = ax.get_legend_handles_labels()
    unique = dict(zip(labels, handles))
    if unique:
        ax.legend(unique.values(), unique.keys(), title="Density window classes")

    ax.set_xlabel("Mean density (veh/km)")
    ax.set_ylabel("Speed / maxspeed (ratio)")
    ax.set_xlim(0, 20)
    ax.set_title("Averaged normalized street speeds vs mean density with class highlights")
    ax.grid(True, which="both", linestyle="--", alpha=0.4)

    class_plot_path = OUTPUT_DIR / "drop_speeds_classes.png"
    plt.savefig(class_plot_path, dpi=500)
    plt.tight_layout()
    print(f"Saved plot: {class_plot_path}")
    plt.show()


In [ ]:
# Save selected streets with class assignment
if not classes:
    print("No classes to save; run classification or adjust thresholds.")
else:
    class_rows = []
    for idx, (label, streets) in enumerate(classes.items(), start=1):
        for sid in streets:
            class_rows.append((sid, idx))
    class_map = dict(class_rows)

    edges_copy = edges.copy()
    edges_copy["id_str"] = edges_copy["id"].astype(str)
    edges_copy["class"] = edges_copy["id_str"].map(class_map)
    selected = edges_copy[edges_copy["class"].notna()].copy()

    selected_path = OUTPUT_DIR / "selected_streets.csv"
    selected.to_csv(selected_path, index=False, sep=";")
    print(f"Saved {len(selected)} selected streets to {selected_path}")
